In [ ]:
import pandas as pnd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression #Самый простой 
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor #Лучший если знать как настраивать
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor #Самый частый
from sklearn.model_selection import GridSearchCV #Позволяет подобрать параметры к модели

In [ ]:
# Рисовать графики сразу же
%matplotlib inline

plt.style.use('ggplot')  # Красивые графики
plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок

In [ ]:
#Форматированные строки - ставим F и фигурные скобки
#Матанализ, Линейная алгебра, Матстаистика, Теория вероятностей, Комбинаторика

intensive = "Обучение в ускоренном формате"
duration = "130 минут"
speaker = "Саша"
profession = "джуниор"
whois = speaker + ", " + profession
print(f"Сегодня у нас {intensive}, проведет {whois}, длительностью {duration} и расскажет о том как рассчитываются налоги")

income = [12,26,43,54]
for i in income:
    tax = 0.13 * i
    print(f"Если Вы получили доход в размере {i}, должны уплатить налог в размере {tax}")
    
plt.plot(income)
plt.show()

#После разведочного анализа
#Преобразование данных в табличку, где текстовые данные превращаются в 0 и 1
trips = pnd.read_excel("trips_data.xlsx", index_col=0) #Датафрейм

trips_trasformed = pnd.get_dummies()(trips, columns['city', 'vacation_preference', 'transport_preference'])

input_data = trips_transformed.drop('target', axis=1) #Данные на основании которых мы хотим научиться
target = trips_transformed.target #То что мы пытаемся предсказать

#Выбираем модель. Настраиваем
model = RandomForestClassifier()

#Теперь обучаем ее
model.fit(input_data, target)

#Передаем пример. Это вызывает список всех заголовков с параметрами 0
{col:[0] for col in input_data.columns}
example = {}
example_dataframe = pnd.DataFrame(data=example)

#А теперь пытаемся посмотреть предсказание
model.predict(example_dataframe)

In [ ]:
#Попробуем предсказать курс доллара
#Качаем данные с ЦБ за три года. Открываем табличку. 
dataframe = pnd.read_excel("RC_F01_08_2017_T01_08_2020.xlsx")

In [ ]:


#Выбираем одну колонку. Конкретно курса
rate = dataframe.curs

#Нужны обучающие примеры чтобы находить закономерность
#Давайте попытаемся предсказать курс на завтра на основе данных за 4 недели
past = 28
lenght = len(rate)
count = lenght - past

print(f"Рассматриваем курс USD с {past} по {lenght}, день, всего {count} дней")

past_days = []
current_day = []
for day in range(past, lenght):
    slc = list(rate[(day-past):day])
    past_days.append(slc) #rate [0:29] and so on adds to list 
    current_day.append(rate[day])    
    
past_columns = []
for i in range(past):
    past_columns.append(f"past_{i}")

X = pnd.DataFrame(data=past_days, columns=past_columns) #Получаем 28 колонок и 709 строчек с обучающими примерами
#То на основе чего делаем предсказание

Y = pnd.Series(current_day, name='target') #То что мы пытаемся предсказать

#Обучающая выборка, "Учебник"
X_train = X[:-10] #Берем все, кроме последних 10 строчек
Y_train = Y[:-10]

#Тестовая выборка, "Экзамен"
X_test = X[-10:]
Y_test = Y[-10:]

lr_model = LinearRegression() #Берем модуль линейной регрессии 
lr_model.fit(X_train, Y_train) #Запускаем обучение
prediction = lr_model.predict(X_test) #Смотрим предсказание обученной МЛ

plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()

mae = mean_absolute_error(prediction, Y_test) #Смотрим процент ошибки - то есть средней разницы между МЛ и РЛ 
print(f"MAE = {mae}")

In [ ]:
#Проверяем насколько каждая из колонок имеет ценность

imp = pnd.Series(lr_model.coef_, name='importance')
cols = pnd.Series(past_columns, name='columns')
df = pnd.DataFrame(imp)
df['imp'] = cols
df

In [ ]:
#Пробуем другой модуль, сравниваем результаты
#Настраиваем модуль.Играем с настройками
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10)
rf_model.fit(X_train, Y_train )
prediction = rf_model.predict(X_test)
plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()
mae = mean_absolute_error(prediction, Y_test)
print(f"MAE = {mae}")



In [ ]:
#Проверяем насколько каждая из колонок имеет ценность
#Видим что конкретная модель использует последние 4 дня, а 94% важен последний день

imp = pnd.Series(rf_model.feature_importances_, name='importance')
cols = pnd.Series(past_columns, name='columns')
df = pnd.DataFrame(imp)
df['imp'] = cols
df

In [ ]:
#Пробуем нейронную сеть
mlp_model = MLPRegressor(max_iter=1000, hidden_layer_sizes=(200,200,200))
mlp_model.fit(X_train, Y_train )
prediction = mlp_model.predict(X_test)
plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()
mae = mean_absolute_error(prediction, Y_test)
print(f"MAE = {mae}")

In [ ]:
#Пробуем нейронную сеть
mlp_model = MLPRegressor(max_iter=1000, hidden_layer_sizes=(200,200,200))
mlp_model.fit(X_train, Y_train )
prediction = mlp_model.predict(X_test)
plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()
mae = mean_absolute_error(prediction, Y_test)
print(f"MAE = {mae}")

In [ ]:
#На основании соседних точек делает вывод кто ты. Видим, что данная модель тут вообще не работает 

knn_model = KNeighborsRegressor(n_neighbors=10)
knn_model.fit(X_train, Y_train )
prediction = knn_model.predict(X_test)
plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()
mae = mean_absolute_error(prediction, Y_test)
print(f"MAE = {mae}")

In [ ]:
#Модель для чайников. Дает результат даже без настроек

gbr_model = GradientBoostingRegressor(n_estimators=1000)
gbr_model.fit(X_train, Y_train )
prediction = gbr_model.predict(X_test)
plt.plot(prediction, label="Prediction") #Сравниваем на графике предсказанный курс и реальный
plt.plot(list(Y_test), label = "Real")
plt.legend()
mae = mean_absolute_error(prediction, Y_test)
print(f"MAE = {mae}")

In [ ]:
#Гридсерч делает поиск по сетке, Указываем модель, интересующие параметры и какую модель считать хорошей
#CV - кросс-валидация.Множество вариантов обучения и тестирования на разных отрезках, со смешением и широким спектром
#Все это гарантирует качество, вне зависимости от того, как мы разбили выборку.Честный и незывисимый способ
#Градиентный спуск в линейной регрессии проверяется средней квадратичной ошибкой.

regressor = GradientBoostingRegressor(max_depth=5, min_samples_split=3)
grid = {
        "n_estimators":[50,100,500],
        "learning_rate":[0.05, 0.1, 0.5]
}

GS = GridSearchCV(regressor, grid, cv=5, scoring='neg_mean_absolute_error') #Задаем обязательные параметры

In [ ]:
#Обучаем гридсерч
GS.fit(X_train, Y_train)
GS.best_score_ #Средний счет по всем выборкам
GS.best_estimator_ #Лучшая подборка
GS.best_params_ #Лучшие параметры